In [ ]:
from autogen import ConversableAgent, LLMConfig, run

# todo: rename autogen.messages to autogen.events
# rename messages to events
# keep all names in autogen.message with deprecation message and import from autogen.events
from autogen.messages.run_events import InputRequestEvent
from autogen.tools import tool

llm_config = LLMConfig.from_json(path="OAI_CONFIG_LIST").where(model=["gpt-4o-mini"], api_type="openai")

llm_config

# Single agent run

In [ ]:
@tool()
def get_meaning_of_life():
    return "The meaning of life is 42."


with llm_config:
    agent = ConversableAgent(name="Alice")

get_meaning_of_life.register_for_llm(agent)

# todo: call tool.register_for_execution(user_proxy) for each tool in agent._tools
response = run(
    agent,
    initial_message="What is the meaning of life?",
    max_turns=5,  # todo: this is ignored now
)


for event in response.events:
    print(f"Got event: {event}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)

In [ ]:
print(response.summary)

# Two agent run

In [ ]:
with llm_config:
    agent_1 = ConversableAgent(name="Jane")
    agent_2 = ConversableAgent(name="Bob")

response = run(agent_1, agent_2, initial_message="have a joke off", max_turns=5)


for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)

# Multi agent run with chat manager

In [ ]:
from autogen.agentchat.groupchat.groupchat import GroupChatManager


@tool()
def submit_plan(plan: str) -> str:
    return f"Plan submitted: {plan}"


with llm_config:
    planner = ConversableAgent(
        name="Planner",
        system_message="You are a planner. Collaborate with teacher and reviewer to create lesson plans.",
    )

    reviewer = ConversableAgent(
        name="Reviewer",
        system_message="You are a reviewer. Review lesson plans against 4th grade curriculum. Provide max 3 changes.",
    )

    teacher = ConversableAgent(
        name="Teacher",
        system_message="You are a teacher. Choose topics and work with planner and reviewer. Say DONE! when finished.",
    )

    chat_manager = GroupChatManager()

submit_plan.register_for_llm(teacher)

response = run(
    planner,
    reviewer,
    teacher,
    initial_message="Create lesson plans for 4th grade.",
    chat_manager=chat_manager,
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)

# Swarm chat manager

In [ ]:
from autogen import AfterWorkOption, ConversableAgent
from autogen.chat_managers import SwarmChatManager  # , RoundRobinChatManager

with llm_config:
    # 1. Create our agents
    planner_message = """You are a classroom lesson planner.
    Given a topic, write a lesson plan for a fourth grade class.
    If you are given revision feedback, update your lesson plan and record it.
    Use the following format:
    <title>Lesson plan title</title>
    <learning_objectives>Key learning objectives</learning_objectives>
    <script>How to introduce the topic to the kids</script>
    """

    planner = ConversableAgent(name="planner_agent", system_message=planner_message)

    reviewer_message = """You are a classroom lesson reviewer.
    You compare the lesson plan to the fourth grade curriculum
    and provide a maximum of 3 recommended changes for each review.
    Make sure you provide recommendations each time the plan is updated.
    """

    reviewer = ConversableAgent(name="reviewer_agent", system_message=reviewer_message)

    teacher_message = """You are a classroom teacher.
    You decide topics for lessons and work with a lesson planner.
    and reviewer to create and finalise lesson plans.
    """

    teacher = ConversableAgent(name="teacher_agent", system_message=teacher_message)

    chat_manager = SwarmChatManager(after_work=AfterWorkOption.SWARM_MANAGER)

response = run(
    planner,
    reviewer,
    teacher,
    initial_message="Today, let's introduce our kids to the solar system.",
    chat_manager=chat_manager,
    max_turns=5,
)

for event in response.events:
    print(f"Got event: {event.type=}, {event=}")
    if isinstance(event, InputRequestEvent):
        event.respond(input(event.prompt))

print(response.summary)

# Sequential chat

# Nested Chat